## Un-supervised thicknees and Rougness measurement from crossection samples

# Method 1

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Load the image
image_path = "soil-layers.jpeg"  # <-- replace with your file name
img = cv2.imread(image_path)

# Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply Canny edge detection
edges = cv2.Canny(gray, 50, 150)

# Horizontal projection to find horizontal transitions
horizontal_projection = np.sum(edges, axis=1)
threshold = np.max(horizontal_projection) * 0.3
boundaries = np.where(horizontal_projection > threshold)[0]

# Filter noise from closely spaced boundaries
clean_boundaries = [boundaries[0]]
for b in boundaries[1:]:
    if b - clean_boundaries[-1] > 10:
        clean_boundaries.append(b)
clean_boundaries.append(img.shape[0])  # add bottom of image

# Compute thickness between clean boundaries
layer_thicknesses = np.diff(clean_boundaries)

# Compute roughness
interface_roughness_std = []
interface_roughness_range = []

for i in range(len(clean_boundaries) - 1):
    y_level = clean_boundaries[i]
    y_profile = []

    for x in range(edges.shape[1]):
        col = edges[:, x]
        y_coords = np.where(col > 0)[0]
        near_y = y_coords[np.abs(y_coords - y_level) < 10]
        if len(near_y) > 0:
            y_profile.append(np.mean(near_y))
        else:
            y_profile.append(np.nan)

    y_profile = np.array(y_profile)
    valid_y = y_profile[~np.isnan(y_profile)]

    if len(valid_y) > 0:
        std_dev = np.std(valid_y)
        y_range = np.max(valid_y) - np.min(valid_y)
    else:
        std_dev = np.nan
        y_range = np.nan

    interface_roughness_std.append(std_dev)
    interface_roughness_range.append(y_range)

# Store results
results_df = pd.DataFrame(
    {
        "Layer": [f"Layer {i + 1}" for i in range(len(layer_thicknesses))],
        "Thickness (px)": layer_thicknesses,
        "Interface Roughness (Std Dev, px)": interface_roughness_std,
        "Interface Roughness (Range, px)": interface_roughness_range,
    }
)

# Print and save
print(results_df)
results_df.to_csv("soil_edge_based_analysis.csv", index=False)

# Visualize
plt.figure(figsize=(12, 6))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
for y in clean_boundaries:
    plt.axhline(y, color="red", linestyle="--")
plt.title("Detected Layer Boundaries")
plt.show()

# Method 2

In [ ]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

image_path = "soil-layers.jpeg"  # Replace with actual path if needed
img = cv2.imread(image_path)

# Resize for faster processing (optional)
resized_img = cv2.resize(img, (img.shape[1], 200))  # keep width, shrink height

# Convert to Lab color space
lab_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2LAB)
pixels = lab_img.reshape(-1, 3)

# KMeans clustering
k = 5  # expected number of layers
kmeans = KMeans(n_clusters=k, random_state=0).fit(pixels)
labels = kmeans.labels_.reshape(resized_img.shape[:2])

# Analyze layers
layer_bounds = []
for cluster_id in range(k):
    ys, xs = np.where(labels == cluster_id)
    if len(ys) > 0:
        layer_bounds.append((cluster_id, np.min(ys), np.max(ys)))

# Sort top to bottom
layer_bounds = sorted(layer_bounds, key=lambda x: x[1])

# Compute thickness and roughness
results = []
for i, (cluster_id, y_min, y_max) in enumerate(layer_bounds):
    thickness = y_max - y_min
    y_std = np.std(np.where(labels[y_min:y_max, :] == cluster_id)[0])
    y_range = y_max - y_min
    results.append(
        {
            "Layer": f"Layer {i + 1}",
            "Thickness (px)": thickness,
            "Interface Roughness (Range, px)": y_range,
            "Interface Roughness (Std Dev, px)": y_std,
        }
    )

# Convert to DataFrame and save
df = pd.DataFrame(results)
print(df)

# Save to CSV
df.to_csv("soil_layer_analysis.csv", index=False)

# Optional: visualize result
plt.imshow(labels, cmap="tab10")
plt.title("Soil Layers by Color Clustering")
plt.show()

# Method 3

In [ ]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

# Load image
image_path = "soil-layers.jpeg"  # Replace with your image path
img = cv2.imread(image_path)

# Resize for faster processing
resized_img = cv2.resize(img, (img.shape[1], 200))

# Convert to Lab color space
lab_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2LAB)

# Average color across each row
mean_color_per_row = np.mean(lab_img, axis=1)
l_channel = mean_color_per_row[:, 0]  # L channel = brightness

# Detect peaks in gradient of brightness
gradient_l = np.gradient(l_channel)
peaks, _ = find_peaks(np.abs(gradient_l), distance=10, prominence=2)

# Add bottom boundary
boundaries = list(peaks)
boundaries.append(resized_img.shape[0])
boundaries = sorted(boundaries)

# Calculate thickness and roughness
thicknesses = np.diff(boundaries)
std_devs, ranges = [], []

for i in range(len(boundaries) - 1):
    segment = lab_img[boundaries[i] : boundaries[i + 1], :, 0]
    row_means = np.mean(segment, axis=1)
    std_devs.append(np.std(row_means))
    ranges.append(np.max(row_means) - np.min(row_means))

# Make DataFrame
df = pd.DataFrame(
    {
        "Layer": [f"Layer {i + 1}" for i in range(len(thicknesses))],
        "Thickness (px)": thicknesses,
        "Interface Roughness (Std Dev, px)": std_devs,
        "Interface Roughness (Range, px)": ranges,
    }
)

# Print and save results
print(df)
df.to_csv("color_gradient_layer_analysis.csv", index=False)

# Plot result
plt.figure(figsize=(12, 6))
plt.imshow(cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB))
for y in boundaries:
    plt.axhline(y, color="cyan", linestyle="--")
plt.title("Detected Layer Boundaries (Color-Gradient Based)")
plt.show()